# 🚀 Lab DataOps: Governança e Qualidade de Dados com Great Expectations

## Objetivos do Laboratório

Neste laboratório prático, você irá:
- Implementar **testes de qualidade de dados** automatizados com **Great Expectations**
- Aplicar as **6 dimensões da qualidade** na prática
- Criar um **pipeline DataOps** com validações profissionais
- Simular cenários reais de **governança de dados**
- Gerar **relatórios de qualidade** automatizados

### Conceitos Aplicados
- **DataOps**: Automação e monitoramento contínuo
- **Governança**: Regras, políticas e responsabilidades
- **Qualidade**: Acurácia, Completude, Consistência, Pontualidade, Unicidade e Validade
- **Great Expectations**: Framework profissional para validação de dados

## 1. Configuração do Ambiente

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import os

# Great Expectations imports
try:
    import great_expectations as gx
    print(f"✅ Great Expectations versão: {gx.__version__}")
except ImportError:
    print("❌ Great Expectations não encontrado. Instalando...")
    !pip install great-expectations==0.18.8 sqlalchemy==1.4.46
    import great_expectations as gx
    print(f"✅ Great Expectations instalado: {gx.__version__}")

print("📊 Ambiente configurado com sucesso!")

## 2. Criação de Dados de Exemplo

In [ ]:
# Dados simulados com problemas de qualidade intencionais
dados_clientes = {
    'id_cliente': [1, 2, 3, 4, 1, 5, 6, 7, 8, 9],  # Duplicata no ID 1
    'nome': ['João Silva', 'Maria Santos', 'Pedro', 'Ana Costa', 'João Silva', '', 'Lucia Oliveira', 'Roberto Lima', 'Fernanda', 'Marcos Pereira'],
    'email': ['joao@email.com', 'maria.santos@gmail.com', 'pedro@invalid', 'ana@email.com', 'joao@email.com', 'carlos@email.com', 'lucia@email.com', 'roberto@email.com', None, 'marcos@email.com'],
    'telefone': ['11999887766', '11888776655', '119999', '11777665544', '11999887766', '11666554433', '11555443322', '11444332211', '11333221100', '11222110099'],
    'data_cadastro': ['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-01-15', '2023-05-12', '2022-12-01', '2023-06-18', '2023-07-22', '2023-08-30'],
    'status': ['Ativo', 'Ativo', 'Inativo', 'Ativo', 'Ativo', 'Pendente', 'Ativo', 'Cancelado', 'Ativo', 'Ativo'],
    'idade': [25, 32, 150, 28, 25, None, 45, 38, 29, -5],  # Problemas: 150 (impossível), None, -5 (negativa)
    'estado': ['SP', 'RJ', 'MG', 'SP', 'SP', 'RS', 'BA', 'PR', 'SC', 'GO']
}

df_clientes = pd.DataFrame(dados_clientes)
print("📋 Dataset de clientes criado com problemas de qualidade intencionais")
print(f"📊 Total de registros: {len(df_clientes)}")
display(df_clientes)

## 3. Configuração do Great Expectations

In [ ]:
# Configurar diretório do Great Expectations
project_dir = "/tmp/gx_lab_dataops"
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

# Inicializar Data Context do Great Expectations
try:
    context = gx.get_context()
    print("📁 Contexto Great Expectations carregado")
except:
    context = gx.get_context(project_root_dir=project_dir)
    print("📁 Novo contexto Great Expectations criado")

print(f"✅ Great Expectations configurado")
print(f"📊 Dataset preparado: {len(df_clientes)} registros")

## 4. Criação de Expectativas de Qualidade

In [ ]:
# Criar validator usando a API simplificada
try:
    validator = context.sources.pandas_default.read_dataframe(df_clientes)
    print("✅ Validator criado com sucesso")
except Exception as e:
    print(f"⚠️ Erro: {e}")
    print("🔧 Tentando método alternativo...")
    # Método alternativo mais básico
    validator = gx.validator.validator.Validator(
        execution_engine=gx.execution_engine.PandasExecutionEngine(),
        interactive_evaluation=True
    )
    validator.load_batch_list([gx.core.batch.Batch(data=df_clientes)])
    print("✅ Validator criado com método alternativo")

## 5. Definindo Expectativas das 6 Dimensões da Qualidade

In [ ]:
print("🎯 CRIANDO EXPECTATIVAS DAS 6 DIMENSÕES DA QUALIDADE")
print("=" * 60)

# 1. COMPLETUDE (Completeness) - Campos não podem ser nulos
print("\n📊 1. COMPLETUDE - Campos obrigatórios")
validator.expect_column_values_to_not_be_null("id_cliente")
validator.expect_column_values_to_not_be_null("nome")
validator.expect_column_values_to_not_be_null("email")
print("   ✅ Expectativas de completude criadas")

# 2. UNICIDADE (Uniqueness) - Valores únicos
print("\n🔑 2. UNICIDADE - Chaves únicas")
validator.expect_column_values_to_be_unique("id_cliente")
validator.expect_column_values_to_be_unique("email")
print("   ✅ Expectativas de unicidade criadas")

# 3. VALIDADE (Validity) - Formatos e domínios
print("\n✅ 3. VALIDADE - Formatos e domínios")
# Email válido
validator.expect_column_values_to_match_regex(
    "email", 
    r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$"
)
# Idade válida (0-120)
validator.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
# Status válido
validator.expect_column_values_to_be_in_set("status", ["Ativo", "Inativo", "Pendente"])
print("   ✅ Expectativas de validade criadas")

# 4. CONSISTÊNCIA (Consistency) - Regras de negócio
print("\n🔄 4. CONSISTÊNCIA - Regras de negócio")
# Telefone deve ter 11 dígitos
validator.expect_column_value_lengths_to_equal("telefone", 11)
# Data de cadastro não pode ser futura
validator.expect_column_values_to_be_dateutil_parseable("data_cadastro")
print("   ✅ Expectativas de consistência criadas")

# 5. PONTUALIDADE (Timeliness) - Dados atuais
print("\n⏰ 5. PONTUALIDADE - Dados atuais")
# Data de cadastro deve ser dos últimos 2 anos
data_limite = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
validator.expect_column_values_to_be_between(
    "data_cadastro", 
    min_value=data_limite, 
    max_value=datetime.now().strftime("%Y-%m-%d")
)
print("   ✅ Expectativas de pontualidade criadas")

# 6. ACURÁCIA (Accuracy) - Dados corretos
print("\n🎯 6. ACURÁCIA - Dados corretos")
# Nome não pode ser vazio
validator.expect_column_values_to_not_match_regex("nome", r"^\s*$")
# Estado deve ser sigla válida (2 caracteres)
validator.expect_column_value_lengths_to_equal("estado", 2)
print("   ✅ Expectativas de acurácia criadas")

print("\n💾 Todas as expectativas criadas com sucesso!")

## 6. Execução das Validações

In [ ]:
# Executar validações
print("🔍 EXECUTANDO VALIDAÇÕES DE QUALIDADE")
print("=" * 50)

# Validar os dados
validation_result = validator.validate()

# Analisar resultados
print(f"\n📊 RESULTADOS DA VALIDAÇÃO:")
print(f"   Total de expectativas: {validation_result.statistics['evaluated_expectations']}")
print(f"   Expectativas que passaram: {validation_result.statistics['successful_expectations']}")
print(f"   Expectativas que falharam: {validation_result.statistics['unsuccessful_expectations']}")
print(f"   Taxa de sucesso: {validation_result.statistics['success_percent']:.1f}%")

# Status geral
if validation_result.success:
    print("\n✅ VALIDAÇÃO PASSOU - Dados dentro dos padrões de qualidade")
else:
    print("\n❌ VALIDAÇÃO FALHOU - Problemas de qualidade detectados")
    print("\n🚨 PROBLEMAS ENCONTRADOS:")
    
    for result in validation_result.results:
        if not result.success:
            expectation_type = result.expectation_config.expectation_type
            column = result.expectation_config.kwargs.get('column', 'N/A')
            print(f"   ❌ {expectation_type} - Coluna: {column}")
            if 'partial_unexpected_count' in result.result:
                print(f"      Registros com problema: {result.result['partial_unexpected_count']}")

## 7. Correção de Dados e Re-validação

In [ ]:
def corrigir_dados_qualidade(df):
    """
    Aplica correções baseadas nos resultados das validações
    """
    print("🔧 APLICANDO CORREÇÕES DE QUALIDADE")
    print("=" * 50)
    
    df_corrigido = df.copy()
    
    # 1. Remover duplicatas (manter primeira ocorrência)
    antes = len(df_corrigido)
    df_corrigido = df_corrigido.drop_duplicates(subset=['id_cliente'], keep='first')
    print(f"✅ Duplicatas removidas: {antes - len(df_corrigido)} registros")
    
    # 2. Corrigir idades inválidas
    df_corrigido.loc[(df_corrigido['idade'] < 0) | (df_corrigido['idade'] > 120), 'idade'] = None
    print("✅ Idades inválidas corrigidas")
    
    # 3. Padronizar status
    df_corrigido.loc[df_corrigido['status'] == 'Cancelado', 'status'] = 'Inativo'
    print("✅ Status padronizados")
    
    # 4. Remover registros com campos críticos vazios
    antes = len(df_corrigido)
    df_corrigido = df_corrigido.dropna(subset=['nome', 'email'])
    df_corrigido = df_corrigido[df_corrigido['nome'].str.strip() != '']
    print(f"✅ Registros com campos críticos vazios removidos: {antes - len(df_corrigido)} registros")
    
    print(f"\n📊 Registros após correção: {len(df_corrigido)}")
    return df_corrigido

# Aplicar correções
df_clientes_corrigido = corrigir_dados_qualidade(df_clientes)

print("\n📋 Dados após correção:")
display(df_clientes_corrigido)

## 8. Re-validação com Dados Corrigidos

In [ ]:
print("🔄 RE-VALIDAÇÃO COM DADOS CORRIGIDOS")
print("=" * 50)

# Criar novo validator com dados corrigidos
try:
    validator_corrigido = context.sources.pandas_default.read_dataframe(df_clientes_corrigido)
    print("✅ Novo validator criado")
except Exception as e:
    print(f"⚠️ Usando método alternativo: {str(e)[:50]}...")
    validator_corrigido = gx.validator.validator.Validator(
        execution_engine=gx.execution_engine.PandasExecutionEngine(),
        interactive_evaluation=True
    )
    validator_corrigido.load_batch_list([gx.core.batch.Batch(data=df_clientes_corrigido)])

# Aplicar as mesmas expectativas
print("🎯 Aplicando expectativas aos dados corrigidos...")

# Completude
validator_corrigido.expect_column_values_to_not_be_null("id_cliente")
validator_corrigido.expect_column_values_to_not_be_null("nome")
validator_corrigido.expect_column_values_to_not_be_null("email")

# Unicidade
validator_corrigido.expect_column_values_to_be_unique("id_cliente")
validator_corrigido.expect_column_values_to_be_unique("email")

# Validade
validator_corrigido.expect_column_values_to_match_regex("email", r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$")
validator_corrigido.expect_column_values_to_be_between("idade", min_value=0, max_value=120)
validator_corrigido.expect_column_values_to_be_in_set("status", ["Ativo", "Inativo", "Pendente"])

# Consistência
validator_corrigido.expect_column_value_lengths_to_equal("telefone", 11)
validator_corrigido.expect_column_values_to_be_dateutil_parseable("data_cadastro")

# Pontualidade
data_limite = (datetime.now() - timedelta(days=730)).strftime("%Y-%m-%d")
validator_corrigido.expect_column_values_to_be_between("data_cadastro", min_value=data_limite, max_value=datetime.now().strftime("%Y-%m-%d"))

# Acurácia
validator_corrigido.expect_column_values_to_not_match_regex("nome", r"^\s*$")
validator_corrigido.expect_column_value_lengths_to_equal("estado", 2)

# Executar validação
validation_result_corrigido = validator_corrigido.validate()

# Comparar resultados
print("\n📊 COMPARAÇÃO DE RESULTADOS")
print("=" * 40)
print(f"DADOS ORIGINAIS:")
print(f"   Taxa de sucesso: {validation_result.statistics['success_percent']:.1f}%")
print(f"   Expectativas que falharam: {validation_result.statistics['unsuccessful_expectations']}")

print(f"\nDADOS CORRIGIDOS:")
print(f"   Taxa de sucesso: {validation_result_corrigido.statistics['success_percent']:.1f}%")
print(f"   Expectativas que falharam: {validation_result_corrigido.statistics['unsuccessful_expectations']}")

if validation_result_corrigido.success:
    print("\n🎉 SUCESSO! Dados corrigidos passaram em todas as validações")
else:
    print("\n⚠️ Ainda existem problemas nos dados corrigidos")
    
melhoria = validation_result_corrigido.statistics['success_percent'] - validation_result.statistics['success_percent']
print(f"\n🎯 Melhoria alcançada: +{melhoria:.1f} pontos percentuais")

## 9. Geração de Relatórios de Qualidade

In [ ]:
# Gerar Data Docs (relatórios HTML)
print("📈 GERANDO RELATÓRIOS DE QUALIDADE (DATA DOCS)")
print("=" * 50)

try:
    # Construir Data Docs
    context.build_data_docs()
    
    # Obter URLs dos relatórios
    data_docs_sites = context.get_docs_sites_urls()
    
    print("✅ Relatórios de qualidade gerados com sucesso!")
    print("\n📊 RELATÓRIOS DISPONÍVEIS:")
    
    for site_name, url in data_docs_sites.items():
        print(f"   📋 {site_name}: {url}")
        
    print("\n💡 Abra os URLs acima no navegador para visualizar os relatórios detalhados")
    
except Exception as e:
    print(f"⚠️ Erro ao gerar Data Docs: {str(e)}")

# Resumo executivo
print("\n📋 RESUMO EXECUTIVO DE QUALIDADE")
print("=" * 50)
print(f"📊 Dataset Original:")
print(f"   - Registros: {len(df_clientes)}")
print(f"   - Taxa de qualidade: {validation_result.statistics['success_percent']:.1f}%")
print(f"   - Status: {'✅ APROVADO' if validation_result.success else '❌ REPROVADO'}")

print(f"\n📊 Dataset Corrigido:")
print(f"   - Registros: {len(df_clientes_corrigido)}")
print(f"   - Taxa de qualidade: {validation_result_corrigido.statistics['success_percent']:.1f}%")
print(f"   - Status: {'✅ APROVADO' if validation_result_corrigido.success else '❌ REPROVADO'}")

melhoria = validation_result_corrigido.statistics['success_percent'] - validation_result.statistics['success_percent']
print(f"\n🎯 Melhoria alcançada: +{melhoria:.1f} pontos percentuais")

## 10. Conclusões

### 🎯 O que Aprendemos com Great Expectations

Neste laboratório, implementamos um **pipeline DataOps profissional** usando Great Expectations:

#### ✅ Vantagens do Great Expectations:
- **Padronização**: Expectativas consistentes e reutilizáveis
- **Automação**: Validações automatizadas
- **Relatórios**: Data Docs profissionais em HTML
- **Integração**: Fácil integração com pipelines de dados
- **Versionamento**: Controle de versão das expectativas

#### ✅ 6 Dimensões Implementadas:
- **Completude**: `expect_column_values_to_not_be_null`
- **Unicidade**: `expect_column_values_to_be_unique`
- **Validade**: `expect_column_values_to_match_regex`, `expect_column_values_to_be_between`
- **Consistência**: `expect_column_value_lengths_to_equal`
- **Pontualidade**: `expect_column_values_to_be_between` (datas)
- **Acurácia**: `expect_column_values_to_not_match_regex`

### 💡 Lições Principais

> **"Great Expectations transforma validações ad-hoc em um sistema profissional de qualidade"**

- **Expectativas são código**: Versionáveis e testáveis
- **Automação é fundamental**: Elimina trabalho manual
- **Relatórios são essenciais**: Facilitam comunicação
- **Integração é chave**: Funciona com qualquer pipeline